In [2]:
import numpy as np
from datasets import load_dataset

# Load dataset
dataset = load_dataset("sms_spam")

# Check structure
print(dataset)
print(dataset["train"].column_names)
print(set(dataset["train"]["label"]))


c:\Users\Blue City\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})
['sms', 'label']
{0, 1}


In [3]:
label_map = {0: "ham", 1: "spam"}
id_map = {"ham": 0, "spam": 1}


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(
        example["sms"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)


In [5]:
tokenized_dataset = tokenized_dataset.remove_columns(["sms"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

In [6]:
train_data = tokenized_dataset["train"].shuffle(seed=42)

train_dataset = train_data.select(range(4500))
eval_dataset = train_data.select(range(4500, len(train_data)))


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label=label_map,
    label2id=id_map
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",          
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",     
    greater_is_better=True
)


In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


C:\Users\Blue City\AppData\Local\Temp\ipykernel_7712\887494177.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


In [ ]:
trainer.save_model("./spam_model")
tokenizer.save_pretrained("./spam_model")

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="./spam_model",
    tokenizer="./spam_model"
)

print(classifier("Congratulations! You won a free prize"))
print(classifier("Hey, are we meeting tomorrow?"))
